## Stage_1 experiment

Here I investigated how well devolearn extract centroid data from various microscopy video datasets.

1. create environment
2. run ```sh install_dependencies.sh``` to install dependencies.


### import packages

In [30]:
import sys
from pathlib import Path
sys.path.append('../')
sys.path.append('../devolearn')

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import torch as th
import devolearn
DATAPATH = '../data'
PREDPATH = os.path.join(DATAPATH, 'pred')

from copy import deepcopy
from importlib import reload
# import devograph.datasets.datasets as data

%matplotlib inline
%config InlineBackend.figure_format='retina'

### trying to extract centroids from videos of EPIC datasets and ipsm dataset - failed.

In [32]:
from devolearn import cell_membrane_segmentor

In [33]:
# didn't work well
segmentor = cell_membrane_segmentor()
EPIC_prefix = "EPIC/20081016_alr-1_10A2_3_L1"
ipsm_prefix = "ipsm/sm01"
EPIC_out_dir = os.path.join(PREDPATH, EPIC_prefix)
ipsm_out_dir = os.path.join(PREDPATH, ipsm_prefix)
os.makedirs(EPIC_out_dir, exist_ok=True)
os.makedirs(ipsm_out_dir, exist_ok=True)
df = segmentor.predict_from_video(video_path = os.path.join(DATAPATH, EPIC_prefix + '.mov'), centroid_mode = True, save_folder = EPIC_out_dir)
df = segmentor.predict_from_video(video_path = os.path.join(DATAPATH, ipsm_prefix + '.mov'), centroid_mode = True, save_folder = ipsm_out_dir)

saving predictions: 100%|██████████| 450/450 [01:51<00:00,  4.03it/s]


### replication - trying to extract centroids from videos of sample_data of devolearn's cell_membrane_segmentor

In [34]:
# worked
segmentor = cell_membrane_segmentor()
sample_data_dir = '../devolearn/devolearn/tests/sample_data'
sample_data_prefix = 'videos/seg_sample'
sample_data_out_dir = os.path.join(PREDPATH, sample_data_prefix)
os.makedirs(sample_data_out_dir, exist_ok=True)
df = segmentor.predict_from_video(video_path = os.path.join(sample_data_dir, sample_data_prefix + '.mov'), centroid_mode = True, save_folder = sample_data_out_dir)

saving predictions: 100%|██████████| 421/421 [01:45<00:00,  3.99it/s]


In [35]:
df.to_csv(os.path.join(sample_data_out_dir, 'centroids.csv'))

### replication - trying to extract centroids from image of sample_data of devolearn's cell_nucleus_segmentor

In [36]:
from devolearn import cell_nucleus_segmentor
segmentor = cell_nucleus_segmentor()
sample_data_dir = '../devolearn/devolearn/tests/sample_data'
sample_data_prefix = 'images/nucleus_seg_sample'
sample_data_out_dir = os.path.join(PREDPATH, sample_data_prefix)
image, centroids = segmentor.predict(image_path = os.path.join(sample_data_dir, sample_data_prefix + ".png"), centroid_mode = True)

### trying to extract centroids from image of sample_data of devolearn's cell_nucleus_segmentor

In [37]:
from devolearn import cell_nucleus_segmentor
segmentor = cell_nucleus_segmentor()

EPIC_prefix = "EPIC/20081016_alr-1_10A2_3_L1"
ipsm_prefix = "ipsm/sm01"
EPIC_out_dir = os.path.join(PREDPATH, EPIC_prefix)
ipsm_out_dir = os.path.join(PREDPATH, ipsm_prefix)
os.makedirs(EPIC_out_dir, exist_ok=True)
os.makedirs(ipsm_out_dir, exist_ok=True)
df = segmentor.predict_from_video(video_path = os.path.join(DATAPATH, EPIC_prefix + '.mov'), centroid_mode = True, save_folder = EPIC_out_dir)

saving predictions: 100%|██████████| 236/236 [00:58<00:00,  4.01it/s]


In [38]:
df.to_csv(os.path.join(EPIC_out_dir, 'centroids.csv'))